In [1]:
!pip install --upgrade --quiet langchain-community langchain-openai neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = ""
# os.environ["NEO4J_URI"] = ""
# os.environ["NEO4J_USERNAME"] = ""
# os.environ["NEO4J_PASSWORD"] = ""

In [3]:
!pip install pandas

In [5]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=r"url",
    username="username",
    password=r"password",
)

In [6]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Disease {identifier: STRING, kind: STRING, name: STRING}
Gene {identifier: INTEGER, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING, description: STRING, chromosome: STRING}
Anatomy {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Compound {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Symptom {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Pathway {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Molecular Function {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Cellular Component {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Biological Process {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Principal Investigators {Prin

In [7]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key='api_key')
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [16]:
prompt_template = """
    make sure the keywords in the prompt are NOT case sensitive. make sure the labels with more than one word in the query are enclosed between ``.don't forget to return values (not keys) the result of the query. also remember to answer each question asked. make sure to pay careful attention to the directions of the relationships, while creating the cypher queries. return the full context of the returned result in human readable language.
    disease: investigated in -> study
    study: tests -> interventions
    interventions are key-value pairs in string format that have the following keys: DRUG, PROCEDURE, DEVICE, BEHAVIOUR, DIAGNOSTIC_TEST, BIOLOGICAL, RADIATION, COMBINATION, PRODUCT, GENETIC, DIETARY_SUPPLEMENT, OTHER. If the query is asking about drugs, then look for the drug here.
    disease: associates/upregulates/downregulates -> gene
    studies: led by -> principal investigators
    disease: studied by -> principal investigators
    compounds: treat/palliate -> disease
    disease: investigated in -> phase
    phases in the knowledge graph are in capital letters with no gap between the word phase and the phase number, such as PHASE1, PHASE2, PHASE3, PHASE4
    principal investigators in the graph are under the label 'Principal Investigators'. For questions about principal investigators, the final return statement should be 'RETURN pi'. disease is studied by principal investigators.
    consider all relations bidirectionally
    consider study title as `Study Title` itself. Dont put any _ between them
"""

query = 'who are the principal investigators of Vitiligo?'

In [17]:


%%capture cellop
response = chain.invoke({"query": (prompt_template+query)})

# response['result']

In [18]:
print("cellop: ", cellop())

parts = cellop.stdout.split('Full Context:')[1]
parts

print("parts: ", parts)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: "Vitiligo"})-[:studied_by]->(pi:`Principal Investigators`)
RETURN pi

Full Context:
[{'pi': {'Principal Investigators 1': 'Pfizer .gov Call Center'}}, {'pi': {'Principal Investigators 1': 'Orest Szczurko'}}, {'pi': {'Principal Investigators 1': 'Jean-Marie Naeyaert'}}, {'pi': {'Principal Investigators 1': 'John E Harris'}}, {'pi': {'Principal Investigators 1': 'Doha S Mohamed'}}, {'pi': {'Principal Investigators 1': 'Samia Esmat'}}, {'pi': {'Principal Investigators 1': 'Julien'}}, {'pi': {'Principal Investigators 1': 'Mor Pavlovski'}}, {'pi': {'Principal Investigators 1': 'Andrew McGovern'}}, {'pi': {'Principal Investigators 1': 'Gabriella Pravatà'}}]

> Finished chain.
cellop:  None
parts:  
[{'pi': {'Principal Investigators 1': 'Pfizer .gov Call Center'}}, {'pi': {'Principal Investigators 1': 'Orest Szczurko'}}, {'pi': {'Principal Investigators 1': 'Jean-Marie Naeyaert'}}, {'pi': {'Principal

In [19]:
# response['result']

In [20]:
from openai import OpenAI

OPENAI_API_KEY = 'api_key'
client = OpenAI(api_key=OPENAI_API_KEY)
prompt = "I am going to give you a question I asked, and the answer to the question which I already have. If the answer is not blank, then return the answer in human readable natural language form. If the answer is blank, then explain that the query made to the knowledge graph did not return any relevant answers. Here is the question: "+query+ "Here is the answer: "+parts

completion = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": "You are a clinical research assistant"},
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='The principal investigators of Vitiligo include:\n\n1. Pfizer .gov Call Center\n2. Orest Szczurko\n3. Jean-Marie Naeyaert\n4. John E Harris\n5. Doha S Mohamed\n6. Samia Esmat\n7. Julien\n8. Mor Pavlovski\n9. Andrew McGovern\n10. Gabriella Pravatà', role='assistant', function_call=None, tool_calls=None)
